<a href="https://colab.research.google.com/github/sebdavid3/Data-Structures-2/blob/main/Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyvis
!pip install graphviz

In [ ]:
import pandas as pd
import folium as fl
from pyvis.network import Network
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from geopy.distance import geodesic

In [ ]:
data = pd.read_csv("flights_final.csv")

In [ ]:
data = data[~data[['Source Airport Code', 'Destination Airport Code']].duplicated(keep='first')].reset_index(drop=True)
data

In [ ]:
data_origin = data[data.columns[:6]].drop_duplicates().reset_index(drop=True)
data_origin.columns = ['Code', 'Name', 'City', 'Country', 'Latitude', 'Longitude']
data_origin.head()

,Code,Name,City,Country,Latitude,Longitude
0,COK,Cochin International Airport,Kochi,India,10.152000,76.401901
1,GIG,Rio Galeão – Tom Jobim International Airport,Rio De Janeiro,Brazil,-22.809999,-43.250557
2,TSF,Treviso-Sant'Angelo Airport,Treviso,Italy,45.648399,12.194400
3,CDG,Charles de Gaulle International Airport,Paris,France,49.012798,2.550000
4,CGH,Congonhas Airport,Sao Paulo,Brazil,-23.626110,-46.656387


In [ ]:
data_destination = data[data.columns[6:]].drop_duplicates().reset_index(drop=True)
data_destination.columns = ['Code', 'Name', 'City', 'Country', 'Latitude', 'Longitude']
data_destination.head()

,Code,Name,City,Country,Latitude,Longitude
0,SHJ,Sharjah International Airport,Sharjah,United Arab Emirates,25.328600,55.517200
1,BSB,Presidente Juscelino Kubistschek International...,Brasilia,Brazil,-15.869167,-47.920834
2,BRI,Bari Karol Wojtyła Airport,Bari,Italy,41.138901,16.760599
3,FLR,Peretola Airport,Florence,Italy,43.810001,11.205100
4,CGB,Marechal Rondon Airport,Cuiaba,Brazil,-15.652900,-56.116699


In [ ]:
airports = pd.concat([data_origin, data_destination]).drop_duplicates().reset_index(drop=True)
airports.head()

,Code,Name,City,Country,Latitude,Longitude
0,COK,Cochin International Airport,Kochi,India,10.152000,76.401901
1,GIG,Rio Galeão – Tom Jobim International Airport,Rio De Janeiro,Brazil,-22.809999,-43.250557
2,TSF,Treviso-Sant'Angelo Airport,Treviso,Italy,45.648399,12.194400
3,CDG,Charles de Gaulle International Airport,Paris,France,49.012798,2.550000
4,CGH,Congonhas Airport,Sao Paulo,Brazil,-23.626110,-46.656387


In [ ]:
map = fl.Map()
lista = airports.values.tolist()
for row in lista:
    fl.Marker([row[-2], row[-1]], popup = ', '.join(row[:-2])).add_to(map)
map

In [ ]:
G=nx.Graph()
datan = airports["Code"]
G.add_nodes_from(datan)# add nodes

for i in data.index:
     distance = geodesic((data['Destination Airport Latitude'][i], data['Destination Airport Longitude'][i]), (data['Source Airport Latitude'][i], data['Source Airport Longitude'][i])).kilometers
     G.add_edge(data['Source Airport Code'][i],data['Destination Airport Code'][i], length= 1000, weight = distance)


In [ ]:
#1 Dado el codigo del aeropuerto

#Mostrar información
cod = "BAQ" # <---------------- Entrada

info = airports[airports['Code'] == cod]
info

,Code,Name,City,Country,Latitude,Longitude
1604,BAQ,Ernesto Cortissoz International Airport,Barranquilla,Colombia,10.8896,-74.7808


In [ ]:
class LL():
  def __init__(self, cod, dist):
    self.cod = cod
    self.dist = dist
    self.next = None
    self.prev = None

class Manager():
    def __init__(self):
      self.head = None
      self.tail = None
      self.len = 0

    def add(self, cod, dist):
      node = LL(cod, dist)
      if self.head is None:
        self.head = node
        self.tail = node
        self.len +=1
        return

      if self.len == 10:
        if node.dist > self.tail.dist:
          self.tail = self.tail.prev
          self.tail.next = None
          self.len-=1
        else:
          return
      if node.dist > self.head.dist:
        node.next = self.head
        self.head.prev = node
        self.head = node
        self.len +=1
        return
      nextNode = self.head
      found = True
      while(nextNode != None and found):
        if node.dist > nextNode.dist:
          found = False
        else:
          nextNode = nextNode.next
      if not found:
        node.prev = nextNode.prev
        node.next = nextNode
        node.prev.next = node
        nextNode.prev = node
      else:
        self.tail.next = node
        node.prev = self.tail
        self.tail = node
      self.len +=1

    def show(self):
      res = []
      act = self.head
      while act is not None:
        res.append([act.cod, act.dist])
        act = act.next
      return res



In [ ]:
lejanos = Manager()
list1 = G.nodes
list2=[]
for i in list1:
  try:
    distancia = nx.dijkstra_path_length(G, cod, i)
  except:
    distance = float('inf')
  lejanos.add(i, distancia)
top10 = np.array(lejanos.show())

In [ ]:
#Mostrar la información de los 10 aeropuertos cuyos caminos mínimos desde el vértice dado sean los más largos.
filas_top10 = airports[airports['Code'].isin(np.array(top10)[:, 0])].reset_index(drop=True)
filas_top10["Distance"] = top10[:, 1]
filas_top10

,Code,Name,City,Country,Latitude,Longitude,Distance
0,DJJ,Sentani International Airport,Jayapura,Indonesia,-2.57695,140.516007,23155.364518233197
1,XCH,Christmas Island Airport,Christmas Island,Christmas Island,-10.45060,105.690002,22208.086827991363
2,BIK,Frans Kaisiepo Airport,Biak,Indonesia,-1.19002,136.108002,22169.713577337265
3,MKQ,Mopah Airport,Merauke,Indonesia,-8.52029,140.417999,21808.713672943522
4,OKL,Oksibil Airport,Oksibil,Indonesia,-4.90710,140.627700,21792.56103901257
5,TIM,Moses Kilangin Airport,Timika,Indonesia,-4.52828,136.886993,21550.749148639545
6,DCN,RAAF Base Curtin,Derby,Australia,-17.58140,123.828003,21342.32562456517
7,CCK,Cocos (Keeling) Islands Airport,Cocos Keeling Island,Cocos (Keeling) Islands,-12.18830,96.833900,21158.918486517894
8,NBX,Nabire Airport,Nabire,Indonesia,-3.36818,135.496002,21124.126401305843
9,WMX,Wamena Airport,Wamena,Indonesia,-4.10251,138.957001,21018.162431212713


In [ ]:
#2.

#NODO INICIAL - NODO FINAL

nodo1, nodo2 = "BAQ", "DJJ" # <---------- Entradas
try:
  camino = list(nx.dijkstra_path (G, nodo1, nodo2)) #camino minimo de un punto inicial y final
except:
  print(f"No existe un camino entre {nodo1} y {nodo2}")
camino

['BAQ',
 'MIA',
 'ATL',
 'ICN',
 'BKI',
 'TWU',
 'TRK',
 'BPN',
 'MDC',
 'SOQ',
 'MKW',
 'DJJ']

In [ ]:
map = fl.Map()

airAux = airports[airports['Code'].isin(camino)]
air = airAux.set_index('Code').reindex(camino).reset_index()
lista = air.values.tolist()
for row in lista:
  fl.Marker([row[-2], row[-1]], popup = ', '.join(row[:-2])).add_to(map)
for i in range(0, len(lista)-1):
    fl.PolyLine([[lista[i][-2], lista[i][-1]],
                     [lista[i+1][-2], lista[i+1][-1]]]).add_to(map)
map